In [202]:
import pandas as pd
import numpy as np
import chardet

In [203]:
def smart_read_csv(file_path) -> pd.DataFrame:
    with open(file_path,'rb') as f:
        res = chardet.detect(f.read(1000))

    return pd.read_csv(file_path, encoding=res['encoding'], encoding_errors='replace')

try:

    raw_orders = smart_read_csv('../data/orders.csv')
    raw_orders_details = smart_read_csv('../data/order_details.csv')
    raw_pizzas = smart_read_csv('../data/pizzas.csv')
    raw_pizza_types = smart_read_csv('../data/pizza_types.csv')

    print(f"Lectura exitosa de los archivos csv.")
except Exception as e:
    print(f"Error en la lectura de alguno de los archivos csv: {e}")

Lectura exitosa de los archivos csv.


In [ ]:
dimDate = raw_orders['date'].to_frame(name='date')
dimDate = dimDate.drop_duplicates()
dimDate['date_id'] = pd.to_datetime(dimDate['date'],errors='coerce').dt.strftime('%Y%m%d').astype(int)
dimDate['date'] = pd.to_datetime(dimDate['date'], errors='coerce')
dimDate['year'] = dimDate['date'].dt.year
dimDate['month'] = dimDate['date'].dt.strftime('%m')
dimDate['month_name_eng'] = dimDate['date'].dt.month_name()
dimDate['month_name_spa'] = dimDate['date'].dt.month_name(locale='es_ES')
dimDate['day'] = dimDate['date'].dt.strftime('%d')
dimDate['day_name_eng'] = dimDate['date'].dt.day_name()
dimDate['day_name_spa'] = dimDate['date'].dt.day_name(locale='es_ES')
dimDate['quarter'] = dimDate['date'].dt.quarter
dimDate['is_weekend'] = dimDate['date'].dt.weekday >= 5

In [216]:
dimTime = pd.to_datetime(raw_orders['time'], format='%H:%M:%S', errors='coerce').dt.strftime('%H%M').astype(int).to_frame(name='time_id')
dimTime = dimTime.drop_duplicates()
dimTime['time'] = pd.to_datetime(raw_orders['time'], format='%H:%M:%S', errors='coerce')
dimTime['hour'] = dimTime['time'].dt.hour
dimTime['minutes'] = dimTime['time'].dt.minute
dimTime['time_ampm'] = dimTime['time'].dt.strftime('%I:%M %p')

time_conditions=[
    (dimTime['hour'] >= 0) & (dimTime['hour'] < 6),
    (dimTime['hour'] >= 6) & (dimTime['hour'] < 12),
    (dimTime['hour'] >= 12) & (dimTime['hour'] < 19),
    (dimTime['hour'] >= 19)
]

time_results = ['night','morning', 'afternoon', 'night']

dimTime['moment_day'] = np.select(time_conditions, time_results, default='Unknown')
dimTime['is_rushhour'] = (((dimTime['hour'] >= 6) & (dimTime['hour'] < 9)) | ((dimTime['hour'] >= 18) & (dimTime['hour'] < 20)))
